## Leiden Clustering Cell-Tag

### 7-8

In [6]:
import numpy as np
import anndata as ad
import pandas as pd
import os
import numpy as np
import sys
import scanpy as sc

In [ ]:
# Define the path to the main folder
main_folder = "/Users/apple/Desktop/KB/SCSeq_LineageBarcoding2/SCSeq_LineageBarcoding/SCLineage_ConstrativeLearning/main_semi_test"

# Add it to sys.path
sys.path.append(main_folder)

from LCL_eval_final import LCL_Eval
from collections import Counter

### lambda 0.01

In [7]:
INPUT_DIR  = "/Users/apple/Desktop/KB/data/feat_LCL_2025/cell_tag/Cell_tag_diff_lambda/feat_celltag_lambda001_unlab5_bs50_testAsPenalty"
TRAIN_ANND = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_train.h5ad"
TEST_ANND  = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_test.h5ad"

adata_train = ad.read_h5ad(TRAIN_ANND)
adata_test  = ad.read_h5ad(TEST_ANND)
train_labels = adata_train.obs["clone_id"].to_numpy()
test_labels  = adata_test.obs["clone_id"].to_numpy()

train_emb = np.load(os.path.join(INPUT_DIR, "scBaseEncoderFeat_Z_bs50_tau0.5.npy"))
test_emb  = np.load(os.path.join(INPUT_DIR, "test_embedding.npy"))

tr = adata_train.copy()
te = adata_test.copy()
tr.obsm["X_lcl"] = train_emb
te.obsm["X_lcl"] = test_emb
tr.obs["dataset"] = "train"
te.obs["dataset"] = "test"
adata_all = sc.concat([tr, te], axis=0, join="outer")

In [15]:
# Leiden clustering
sc.pp.neighbors(adata_all, use_rep="X_lcl", n_neighbors=15)
sc.tl.leiden(adata_all, resolution=3, key_added="lcl_leiden")

In [16]:
# 5) cluster‐by‐dataset cross‐tab
ct = (
    adata_all.obs
    .groupby(["lcl_leiden", "dataset"])
    .size()
    .unstack(fill_value=0)
)
ct["total"] = ct.sum(axis=1)
ct = ct.sort_values("total", ascending=False)

# ← fix: wrap the & in parentheses so .sum() applies to the combined mask
n_clusters     = adata_all.obs["lcl_leiden"].nunique()
clusters_both  = ((ct["train"] > 0) & (ct["test"] > 0)).sum()
clusters_train = ((ct["train"] > 0) & (ct["test"] == 0)).sum()
clusters_test  = ((ct["train"] == 0) & (ct["test"] > 0)).sum()

# 6) per‐lineage cluster occupancy
lineage_groups = (
    adata_all.obs[["clone_id", "lcl_leiden"]]
    .drop_duplicates()
    .groupby("clone_id")["lcl_leiden"]
    .nunique()
)
n_lineages           = lineage_groups.shape[0]
lineages_one_cluster = (lineage_groups == 1).sum()
prop_one_cluster     = lineages_one_cluster / n_lineages

# 7) print summary
print(f"Total Leiden clusters:           {n_clusters}")
print(f"Clusters with both train & test:  {clusters_both}")
print(f"Clusters train‐only:             {clusters_train}")
print(f"Clusters test‐only:              {clusters_test}")
print()
print(f"Total lineages:                  {n_lineages}")
print(f"Lineages in exactly one cluster: {lineages_one_cluster} ({prop_one_cluster:.2%})")

Total Leiden clusters:           147
Clusters with both train & test:  40
Clusters train‐only:             105
Clusters test‐only:              2

Total lineages:                  169
Lineages in exactly one cluster: 85 (50.30%)


/var/folders/ql/mhz506t52bjgn30w6f4wqhv40000gn/T/ipykernel_21055/1083807402.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata_all.obs


### lambda 0.1

In [17]:
INPUT_DIR  = "/Users/apple/Desktop/KB/data/feat_LCL_2025/cell_tag/Cell_tag_diff_lambda/feat_celltag_lambda01_unlab5_bs50_testAsPenalty"
TRAIN_ANND = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_train.h5ad"
TEST_ANND  = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_test.h5ad"

adata_train = ad.read_h5ad(TRAIN_ANND)
adata_test  = ad.read_h5ad(TEST_ANND)
train_labels = adata_train.obs["clone_id"].to_numpy()
test_labels  = adata_test.obs["clone_id"].to_numpy()

train_emb = np.load(os.path.join(INPUT_DIR, "scBaseEncoderFeat_Z_bs50_tau0.5.npy"))
test_emb  = np.load(os.path.join(INPUT_DIR, "test_embedding.npy"))

tr = adata_train.copy()
te = adata_test.copy()
tr.obsm["X_lcl"] = train_emb
te.obsm["X_lcl"] = test_emb
tr.obs["dataset"] = "train"
te.obs["dataset"] = "test"
adata_all = sc.concat([tr, te], axis=0, join="outer")

In [18]:
# Leiden clustering
sc.pp.neighbors(adata_all, use_rep="X_lcl", n_neighbors=15)
sc.tl.leiden(adata_all, resolution=3, key_added="lcl_leiden")

In [20]:
# 5) cluster‐by‐dataset cross‐tab
ct = (
    adata_all.obs
    .groupby(["lcl_leiden", "dataset"])
    .size()
    .unstack(fill_value=0)
)
ct["total"] = ct.sum(axis=1)
ct = ct.sort_values("total", ascending=False)

# ← fix: wrap the & in parentheses so .sum() applies to the combined mask
n_clusters     = adata_all.obs["lcl_leiden"].nunique()
clusters_both  = ((ct["train"] > 0) & (ct["test"] > 0)).sum()
clusters_train = ((ct["train"] > 0) & (ct["test"] == 0)).sum()
clusters_test  = ((ct["train"] == 0) & (ct["test"] > 0)).sum()

# 6) per‐lineage cluster occupancy
lineage_groups = (
    adata_all.obs[["clone_id", "lcl_leiden"]]
    .drop_duplicates()
    .groupby("clone_id")["lcl_leiden"]
    .nunique()
)
n_lineages           = lineage_groups.shape[0]
lineages_one_cluster = (lineage_groups == 1).sum()
prop_one_cluster     = lineages_one_cluster / n_lineages

# 7) print summary
print(f"Total Leiden clusters:           {n_clusters}")
print(f"Clusters with both train & test:  {clusters_both}")
print(f"Clusters train‐only:             {clusters_train}")
print(f"Clusters test‐only:              {clusters_test}")
print()
print(f"Total lineages:                  {n_lineages}")
print(f"Lineages in exactly one cluster: {lineages_one_cluster} ({prop_one_cluster:.2%})")

Total Leiden clusters:           133
Clusters with both train & test:  25
Clusters train‐only:             106
Clusters test‐only:              2

Total lineages:                  169
Lineages in exactly one cluster: 82 (48.52%)


/var/folders/ql/mhz506t52bjgn30w6f4wqhv40000gn/T/ipykernel_21055/1083807402.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata_all.obs


### lambda 0.5

In [21]:
INPUT_DIR  = "/Users/apple/Desktop/KB/data/feat_LCL_2025/cell_tag/Cell_tag_diff_lambda/feat_celltag_lambda05_unlab5_bs50_testAsPenalty"
TRAIN_ANND = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_train.h5ad"
TEST_ANND  = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_test.h5ad"

adata_train = ad.read_h5ad(TRAIN_ANND)
adata_test  = ad.read_h5ad(TEST_ANND)
train_labels = adata_train.obs["clone_id"].to_numpy()
test_labels  = adata_test.obs["clone_id"].to_numpy()

train_emb = np.load(os.path.join(INPUT_DIR, "scBaseEncoderFeat_Z_bs50_tau0.5.npy"))
test_emb  = np.load(os.path.join(INPUT_DIR, "test_embedding.npy"))

tr = adata_train.copy()
te = adata_test.copy()
tr.obsm["X_lcl"] = train_emb
te.obsm["X_lcl"] = test_emb
tr.obs["dataset"] = "train"
te.obs["dataset"] = "test"
adata_all = sc.concat([tr, te], axis=0, join="outer")

In [22]:
# Leiden clustering
sc.pp.neighbors(adata_all, use_rep="X_lcl", n_neighbors=15)
sc.tl.leiden(adata_all, resolution=3, key_added="lcl_leiden")

In [23]:
# 5) cluster‐by‐dataset cross‐tab
ct = (
    adata_all.obs
    .groupby(["lcl_leiden", "dataset"])
    .size()
    .unstack(fill_value=0)
)
ct["total"] = ct.sum(axis=1)
ct = ct.sort_values("total", ascending=False)

# ← fix: wrap the & in parentheses so .sum() applies to the combined mask
n_clusters     = adata_all.obs["lcl_leiden"].nunique()
clusters_both  = ((ct["train"] > 0) & (ct["test"] > 0)).sum()
clusters_train = ((ct["train"] > 0) & (ct["test"] == 0)).sum()
clusters_test  = ((ct["train"] == 0) & (ct["test"] > 0)).sum()

# 6) per‐lineage cluster occupancy
lineage_groups = (
    adata_all.obs[["clone_id", "lcl_leiden"]]
    .drop_duplicates()
    .groupby("clone_id")["lcl_leiden"]
    .nunique()
)
n_lineages           = lineage_groups.shape[0]
lineages_one_cluster = (lineage_groups == 1).sum()
prop_one_cluster     = lineages_one_cluster / n_lineages

# 7) print summary
print(f"Total Leiden clusters:           {n_clusters}")
print(f"Clusters with both train & test:  {clusters_both}")
print(f"Clusters train‐only:             {clusters_train}")
print(f"Clusters test‐only:              {clusters_test}")
print()
print(f"Total lineages:                  {n_lineages}")
print(f"Lineages in exactly one cluster: {lineages_one_cluster} ({prop_one_cluster:.2%})")

Total Leiden clusters:           61
Clusters with both train & test:  38
Clusters train‐only:             23
Clusters test‐only:              0

Total lineages:                  169
Lineages in exactly one cluster: 62 (36.69%)


/var/folders/ql/mhz506t52bjgn30w6f4wqhv40000gn/T/ipykernel_21055/1083807402.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata_all.obs


### lambda 1

In [24]:
INPUT_DIR  = "/Users/apple/Desktop/KB/data/feat_LCL_2025/cell_tag/Cell_tag_diff_lambda/feat_celltag_lambda1_unlab5_bs50_testAsPenalty"
TRAIN_ANND = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_train.h5ad"
TEST_ANND  = "/Users/apple/Desktop/KB/data/BiddyData/Biddy_test.h5ad"

adata_train = ad.read_h5ad(TRAIN_ANND)
adata_test  = ad.read_h5ad(TEST_ANND)
train_labels = adata_train.obs["clone_id"].to_numpy()
test_labels  = adata_test.obs["clone_id"].to_numpy()

train_emb = np.load(os.path.join(INPUT_DIR, "scBaseEncoderFeat_Z_bs50_tau0.5.npy"))
test_emb  = np.load(os.path.join(INPUT_DIR, "test_embedding.npy"))

tr = adata_train.copy()
te = adata_test.copy()
tr.obsm["X_lcl"] = train_emb
te.obsm["X_lcl"] = test_emb
tr.obs["dataset"] = "train"
te.obs["dataset"] = "test"
adata_all = sc.concat([tr, te], axis=0, join="outer")

In [25]:
# Leiden clustering
sc.pp.neighbors(adata_all, use_rep="X_lcl", n_neighbors=15)
sc.tl.leiden(adata_all, resolution=3, key_added="lcl_leiden")

In [26]:
# 5) cluster‐by‐dataset cross‐tab
ct = (
    adata_all.obs
    .groupby(["lcl_leiden", "dataset"])
    .size()
    .unstack(fill_value=0)
)
ct["total"] = ct.sum(axis=1)
ct = ct.sort_values("total", ascending=False)

# ← fix: wrap the & in parentheses so .sum() applies to the combined mask
n_clusters     = adata_all.obs["lcl_leiden"].nunique()
clusters_both  = ((ct["train"] > 0) & (ct["test"] > 0)).sum()
clusters_train = ((ct["train"] > 0) & (ct["test"] == 0)).sum()
clusters_test  = ((ct["train"] == 0) & (ct["test"] > 0)).sum()

# 6) per‐lineage cluster occupancy
lineage_groups = (
    adata_all.obs[["clone_id", "lcl_leiden"]]
    .drop_duplicates()
    .groupby("clone_id")["lcl_leiden"]
    .nunique()
)
n_lineages           = lineage_groups.shape[0]
lineages_one_cluster = (lineage_groups == 1).sum()
prop_one_cluster     = lineages_one_cluster / n_lineages

# 7) print summary
print(f"Total Leiden clusters:           {n_clusters}")
print(f"Clusters with both train & test:  {clusters_both}")
print(f"Clusters train‐only:             {clusters_train}")
print(f"Clusters test‐only:              {clusters_test}")
print()
print(f"Total lineages:                  {n_lineages}")
print(f"Lineages in exactly one cluster: {lineages_one_cluster} ({prop_one_cluster:.2%})")

Total Leiden clusters:           53
Clusters with both train & test:  39
Clusters train‐only:             14
Clusters test‐only:              0

Total lineages:                  169
Lineages in exactly one cluster: 62 (36.69%)


/var/folders/ql/mhz506t52bjgn30w6f4wqhv40000gn/T/ipykernel_21055/1083807402.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata_all.obs
